### ¿Que hace este script?
* Calcula para cada AP: trafico.totales.totalConexiones
* Calcula para cada site id: trafico.concurrenciaConexiones

In [26]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [27]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [28]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

In [29]:
indice = parametros.trafico_mintic_concat_index
indice_control = parametros.mintic_control

### Función para generar JSON compatible con ES

In [30]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [6]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR

In [7]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","ip","ap_group","name","status"]  
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [8]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian valores errados de ap group

In [9]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

SE borran duplicados por MAC

In [10]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según estructura del indice final

In [11]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'trafico.macRed','ap_group': 'trafico.apGroup'
                                        , 'ip': 'trafico.IP'
                                        , 'mac' : 'trafico.macRed'
                                        , 'name' : 'trafico.deviceName'
                                        , 'status' : 'trafico.status.macRed'})

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [12]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla['trafico.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['trafico.location']=datos_semilla['trafico.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'trafico.nombreMunicipio'
                                              , 'nombre_departamento' : 'trafico.nombreDepartamento'
                                              , 'nombre_centro_pob': 'trafico.localidad'
                                              , 'nombreSede' : 'trafico.nomCentroDigital'
                                              , 'energiadesc' : 'trafico.sistemaEnergia'
                                              , 'COD_ISO' : 'trafico.codISO'
                                              , 'id_Beneficiario' : 'trafico.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [32]:
indice_control

'dev-control_ejecucion_mintic'

In [31]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["trafico.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_trafico_conexiones"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['trafico.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-11 16:10:00


## Traer datos de ohmyfi-detalleconexiones

El rango de fechas será definido tomando de referencia la ultima fechahora del indice mintic-concat. Campos extaidos:
* fechahora de la cnexión 
* fecha_control es un campo calculado a partir de fechahora. es lo mismo pero con el valor de minuto redondeado a 0.
* lugar_cod clave para asociar con semilla
* mac_usuario asociado al dispositivo que realizó la conexión

In [14]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
        #, request_timeout=300
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

### Se acotan los rangos de fecha por eficiencia

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [15]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_det_conex_completo = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

if datos_det_conex_completo is None or datos_det_conex_completo.empty:
    while (datos_det_conex_completo is None or datos_det_conex_completo.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_det_conex_completo = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [16]:
datos_det_conex_completo.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

### contanto conexiones por lugar y fecha

Para cada centro, y fecha control, se cuenta la cantidad de conexiones. Una vez combinado con semilla, se disponibiliza el filtrado por departamento, municipio y centro de conexión. El campo generado es:
* trafico.totales.totalConexiones

In [17]:
datos_det_conex=datos_det_conex_completo[["fechahora","fecha_control","lugar_cod"]].groupby(['lugar_cod','fecha_control']).agg(['count']).reset_index()
datos_det_conex.columns = datos_det_conex.columns.droplevel(1)

### renombrado de columnas en detalle conexiones

In [18]:
datos_det_conex = datos_det_conex.rename(columns={'fechahora' : 'trafico.totales.totalConexiones','lugar_cod':'site_id'})

### Combinando detalle de conexiones con dispositivos usuario y red

In [19]:
total_docs = 30000000
response = es.search(
    index= parametros.cambium_d_c_index,
    body={
            "_source": ["mac","ap_mac", 'manufacturer',"radio.band",'radio.rx_bytes','radio.tx_bytes'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"mac"
                    }
                  }
                  ]
                }
              }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
fields = {}
for num, doc in enumerate(elastic_docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])

datos_dev_clients = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
datos_dev_clients.drop_duplicates(inplace=True)

* Se renombran campos de clientes
* se incorpora informaciones de clientes a detalle conexiones

In [20]:
datos_dev_clients = datos_dev_clients.rename(columns={'ap_mac' : 'trafico.macRed','mac' : 'mac_usuario'})
datos_det_conex_completo = pd.merge(datos_det_conex_completo,  datos_dev_clients, on='mac_usuario',how='left')
datos_det_conex_completo = pd.merge(datos_det_conex_completo,datos_dev[['trafico.macRed','trafico.apGroup','trafico.IP','trafico.deviceName']],on='trafico.macRed', how='left')

## calculando total de conexiones

tiene la información de semilla, total de conexiones, alarmas, performance de AP y recurrencia de usuarios.
Para obtener un registro unico para cada site_id, ap_group y fecha_control, se usa el data frame con fechas unicas (fechas_semilla)

# Total conexiones a indice

In [21]:
use_these_keys = ['trafico.nomCentroDigital', 
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.nombreDepartamento', 
                  'trafico.codISO', 
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio', 
                  'trafico.idBeneficiario',
                  'trafico.apGroup', 
                  'trafico.IP', 
                  'trafico.deviceName',
                  'trafico.macRed', 
                  'trafico.location', 
                  'trafico.totales.totalConexiones', 
                  'trafico.totales.fechaControl',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.minuto', 
                    'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Conexiones-'+document['trafico.siteID'] + '-' + document['trafico.macRed'] + '-' +document['trafico.totales.fechaControl']}",
                "_source": filterKeys(document),
            }

In [22]:
total_conexiones =  datos_det_conex_completo[["fechahora","fecha_control","lugar_cod"
                                              ,"trafico.macRed",'trafico.apGroup'
                                              ,'trafico.IP', 'trafico.deviceName'
                                             ]].groupby(['lugar_cod','fecha_control'
                                                    ,"trafico.macRed",'trafico.apGroup'
                                                    ,'trafico.IP', 'trafico.deviceName']).agg(['count']).reset_index()
total_conexiones.columns = total_conexiones.columns.droplevel(1)
total_conexiones.rename(columns={'fechahora': 'trafico.totales.totalConexiones'
                                , 'lugar_cod' : 'site_id'}, inplace=True)
mintic_01 = pd.merge(datos_semilla,  total_conexiones, on='site_id',how='inner')

In [21]:
try:
    total_conexiones =  datos_det_conex_completo[["fechahora","fecha_control","lugar_cod"
                                                  ,"trafico.macRed",'trafico.apGroup'
                                                  ,'trafico.IP', 'trafico.deviceName'
                                                 ]].groupby(['lugar_cod','fecha_control'
                                                        ,"trafico.macRed",'trafico.apGroup'
                                                        ,'trafico.IP', 'trafico.deviceName']).agg(['count']).reset_index()
    total_conexiones.columns = total_conexiones.columns.droplevel(1)
    total_conexiones.rename(columns={'fechahora': 'trafico.totales.totalConexiones'
                                    , 'lugar_cod' : 'site_id'}, inplace=True)
    mintic_01 = pd.merge(datos_semilla,  total_conexiones, on='site_id',how='inner')
    
    mintic_01.fillna({'trafico.totales.totalConexiones': 0}, inplace=True)
    mintic_01['trafico.totales.totalConexiones'] = mintic_01['trafico.totales.totalConexiones'].astype(int)
    mintic_01.rename(columns={'site_id': 'trafico.siteID'
                             ,'fecha_control' : 'trafico.totales.fechaControl'}, inplace=True)

    mintic_01["trafico.totales.fecha"] = mintic_01["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    mintic_01["trafico.totales.hora"] = mintic_01["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    mintic_01["trafico.totales.minuto"] = mintic_01["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    mintic_01["trafico.totales.anyo"] = mintic_01["trafico.totales.fecha"].str[0:4]
    mintic_01["trafico.totales.mes"] = mintic_01["trafico.totales.fecha"].str[5:7]
    mintic_01["trafico.totales.dia"] = mintic_01["trafico.totales.fecha"].str[8:10]

    #mintic_01["trafico.causaFalla"] = "Pendiente"
    #mintic_01["trafico.accionTomada"] = "Pendiente"
    #mintic_01["trafico.tiempoDuraFalla"] = "Pendiente"
    mintic_01['nombreDepartamento'] = mintic_01['trafico.nombreDepartamento']
    mintic_01['nombreMunicipio'] = mintic_01['trafico.nombreMunicipio']
    mintic_01['idBeneficiario'] = mintic_01['trafico.idBeneficiario']
    mintic_01['fecha'] = mintic_01['trafico.totales.fecha']
    mintic_01['anyo'] = mintic_01['trafico.totales.anyo']
    mintic_01['mes'] = mintic_01['trafico.totales.mes']
    mintic_01['dia'] = mintic_01['trafico.totales.dia']
    mintic_01['@timestamp'] = now.isoformat()        
#     salida = helpers.bulk(es, doc_generator(mintic_01))
    print("Fecha: ", now,"- Datos Trafico en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- Nada de Trafico para insertar en indice principal:")
    

## Se lee información para usuarios recurrencia

Se calcula y agrega al indice principal:
* trafico.concurrenciaConexiones

Se lee el indice recurrencia de conexiones y se compara con el flujo detalle conexiones para el rango dado. Si cruzan, se suma a la cuenta de recurrentes.

In [23]:
total_docs = 5000000
response = es.search(
    index= parametros.ohmyfi_r_u_index,
    body={
            "_source": ["ultima_conexion", "lugar_cod", "id_usuario"]
    },
    size=total_docs
)
#print(es.info())
elastic_docs = response["hits"]["hits"]
fields = {}
for num, doc in enumerate(elastic_docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])

datos_recurrencia = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

Se cuenta la cantidad de usuarios con mas de una conexión

# Concurrencia usuario a indice

In [25]:
use_these_keys = ['trafico.nomCentroDigital',
                  'trafico.codISO',
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.codISO',
                  'trafico.nombreDepartamento',
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio',
                  'trafico.idBeneficiario',
                  'trafico.totales.fechaControl',
                  'trafico.concurrenciaConexiones',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.minuto',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Concurrencia-' + document['trafico.siteID'] + '-' +document['trafico.totales.fechaControl']}",
                "_source": filterKeys(document),
            }

Fecha:  2021-05-28 20:43:11.384583 - Datos Concurrencia de conexiones en indice principal: 14


In [24]:
try:
    datos_det_conex_completo.rename(columns={'mac_usuario':'id_usuario'}, inplace=True)
    aux_recurrencia=datos_det_conex_completo[['lugar_cod','id_usuario']].groupby(['id_usuario']).agg(['count']).reset_index()
    aux_recurrencia.columns = aux_recurrencia.columns.droplevel(1)
    aux_recurrencia.rename(columns={'lugar_cod': 'contador'}, inplace=True)
    aux_recurrencia = aux_recurrencia.drop(aux_recurrencia[(aux_recurrencia['contador'] < 2)].index)
    datos_recurrencia = pd.merge(datos_det_conex_completo,  aux_recurrencia, on='id_usuario', how='inner')
    datos_recurrencia = datos_recurrencia[['lugar_cod','fecha_control','id_usuario']].groupby(['lugar_cod','fecha_control']).agg(['count']).reset_index()
    datos_recurrencia.columns = datos_recurrencia.columns.droplevel(1)
    datos_recurrencia.rename(columns={'lugar_cod':'site_id'}, inplace=True)
    datos_recurrencia = pd.merge(datos_semilla,  datos_recurrencia, on='site_id', how='inner')
    datos_recurrencia.rename(columns={'site_id':'trafico.siteID'
                                     ,'id_usuario': 'trafico.concurrenciaConexiones'
                                     ,'fecha_control' : 'trafico.totales.fechaControl'}, inplace=True)
    datos_recurrencia.fillna({'trafico.concurrenciaConexiones':0},inplace=True)
    datos_recurrencia.fillna('', inplace=True)
    datos_recurrencia["trafico.totales.fecha"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    datos_recurrencia["trafico.totales.anyo"] = datos_recurrencia["trafico.totales.fecha"].str[0:4]
    datos_recurrencia["trafico.totales.mes"] = datos_recurrencia["trafico.totales.fecha"].str[5:7]
    datos_recurrencia["trafico.totales.dia"] = datos_recurrencia["trafico.totales.fecha"].str[8:10]
    datos_recurrencia["trafico.totales.hora"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    datos_recurrencia["trafico.totales.minuto"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    datos_recurrencia['nombreDepartamento'] = datos_recurrencia['trafico.nombreDepartamento']
    datos_recurrencia['nombreMunicipio'] = datos_recurrencia['trafico.nombreMunicipio']
    datos_recurrencia['idBeneficiario'] = datos_recurrencia['trafico.idBeneficiario']
    datos_recurrencia['fecha'] = datos_recurrencia['trafico.totales.fecha']
    datos_recurrencia['anyo'] = datos_recurrencia['trafico.totales.anyo']
    datos_recurrencia['mes'] = datos_recurrencia['trafico.totales.mes']
    datos_recurrencia['dia'] = datos_recurrencia['trafico.totales.dia']
    
    datos_recurrencia['@timestamp'] = now.isoformat() 
    salida = helpers.bulk(es, doc_generator(datos_recurrencia))
    print("Fecha: ", now,"- Datos Concurrencia de conexiones en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No hay datos de concurrencia de conexiones para insetar en indice principal:")

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [27]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_trafico_conexiones',
        body = { 'jerarquia_trafico_conexiones': 'trafico_conexiones','trafico.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-05-15 10:10:00
